<a href="https://colab.research.google.com/github/BPHC-ACM/Face-Anonymiser/blob/main/notebooks/comfyui_colab_with_manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [3]:
# #@title Environment Setup

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f ".ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/nightly/windows_base_files/run_nvidia_gpu.bat" ] && chmod 755 .ci/nightly/windows_base_files/run_nvidia_gpu.bat
  ![ -f ".ci/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows/update.py" ] && chmod 755 .ci/update_windows/update.py
  ![ -f ".ci/update_windows/update_comfyui.bat" ] && chmod 755 .ci/update_windows/update_comfyui.bat
  ![ -f ".ci/update_windows/README_VERY_IMPORTANT.txt" ] && chmod 755 .ci/update_windows/README_VERY_IMPORTANT.txt
  ![ -f ".ci/update_windows/run_cpu.bat" ] && chmod 755 .ci/update_windows/run_cpu.bat
  ![ -f ".ci/update_windows/run_nvidia_gpu.bat" ] && chmod 755 .ci/update_windows/run_nvidia_gpu.bat

  !git pull

!echo -= Install dependencies =-
!pip3 install accelerate
!pip3 install einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install torchsde
!pip3 install kornia>=0.7.1 spandrel soundfile sentencepiece

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
  ![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
  ![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
  ![ -f "ComfyUI-Manager/node_db/tutorial/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/tutorial/scan.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !echo -= Install custom nodes dependencies =-
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies


Mounting Google Drive...
/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
/content/drive/MyDrive/ComfyUI
-= Updating ComfyUI =-
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 8 (delta 6), reused 8 (delta 6), pack-reused 0 (from 0)
Unpacking objects: 100% (8/8), 972 bytes | 0 bytes/s, done.
From https://github.com/comfyanonymous/ComfyUI
   338c88d3..7f72c07e  feat/api-nodes/meshy -> origin/feat/api-nodes/meshy
 * [new branch]        fix/api-nodes/gemini-image-safety -> origin/fix/api-nodes/gemini-image-safety
Updating 5943fbf4..5ac13725
error: Your local changes to the following files would be overwritten by merge:
	comfy/cli_args.py
Please commit your changes or stash them before you merge.
Aborting
-= Install dependencies =-
Looking in indexes: https://download.pytorch.org/whl/cu121
   ━

Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

In [2]:
# --- SAFE-FM SETUP (InstaFlow + MediaPipe) ---

# 1. Download the "Fast" Model (InstaFlow 0.9B)
# This replaces standard SD1.5 to get 1-step inference
!wget -c https://huggingface.co/XCLiu/instaflow_0_9B_from_sd_1_5/resolve/main/instaflow_0_9b.safetensors -P ./models/checkpoints/

# 2. Download the Face Tracker (CrucibleAI MediaPipe)
# This allows the model to "see" the face mesh
!wget -c https://huggingface.co/CrucibleAI/ControlNetMediaPipeFace/resolve/main/control_mediapipe_face_sd15_v2.safetensors -P ./models/controlnet/

# 3. Install the Preprocessors (Critical for the "MediaPipe Face Mesh" Node)
# We must uncomment and run this to get the "Face Mesh" node in ComfyUI
!cd custom_nodes && git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors; cd comfy_controlnet_preprocessors && python install.py

# --- OPTIONAL BACKUPS (Keep these just in case) ---

# Standard VAE (Good for color correction)
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/

# Standard OpenPose (Fallback if MediaPipe fails)
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -P ./models/controlnet/

print("✅ SAFE-FM Models Downloaded Successfully!")

--2026-01-13 09:49:17--  https://huggingface.co/XCLiu/instaflow_0_9B_from_sd_1_5/resolve/main/instaflow_0_9b.safetensors
Resolving huggingface.co (huggingface.co)... 18.65.25.19, 18.65.25.103, 18.65.25.54, ...
Connecting to huggingface.co (huggingface.co)|18.65.25.19|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2026-01-13 09:49:17 ERROR 404: Not Found.

--2026-01-13 09:49:17--  https://huggingface.co/CrucibleAI/ControlNetMediaPipeFace/resolve/main/control_mediapipe_face_sd15_v2.safetensors
Resolving huggingface.co (huggingface.co)... 18.244.214.123, 18.244.214.111, 18.244.214.57, ...
Connecting to huggingface.co (huggingface.co)|18.244.214.123|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2026-01-13 09:49:18 ERROR 404: Not Found.

Cloning into 'comfy_controlnet_preprocessors'...
fatal: could not read Username for 'https://github.com': No such device or address
/bin/bash: line 1: cd: comfy_controlnet_preprocessors: No such file or d

In [4]:
!pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 11.0 MB/s eta 0:00:00


In [5]:
!pip uninstall -y mediapipe protobuf
!pip install mediapipe==0.10.14 protobuf==4.25.3

Found existing installation: mediapipe 0.10.31
Uninstalling mediapipe-0.10.31:
  Successfully uninstalled mediapipe-0.10.31
Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 22.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.3 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.3 which is incompatible.


In [7]:
# --- SAFE-FM MODEL DOWNLOADER (No Installation Risks) ---
import os

# 1. Base Model (Realistic Vision V5.1 - Corrected Link)
!wget -c "https://huggingface.co/SG161222/Realistic_Vision_V5.1_noVAE/resolve/main/Realistic_Vision_V5.1_fp16-no-ema.safetensors" -P ./models/checkpoints/

# 2. Hyper-SD LoRA (The Speed Engine)
!wget -c "https://huggingface.co/ByteDance/Hyper-SD/resolve/main/Hyper-SD15-1step-lora.safetensors" -P ./models/loras/

# 3. ControlNet MediaPipe (The Face Tracker)
# We try the official link.
url_1 = "https://huggingface.co/CrucibleAI/ControlNetMediaPipeFace/resolve/main/control_v2p_sd15_mediapipe_face.safetensors"
!wget -c {url_1} -P ./models/controlnet/

# 4. Backup ControlNet (Canny)
!wget -c "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors" -P ./models/controlnet/

# 5. Get the Custom Node Code (BUT DO NOT RUN PIP INSTALL)
# We just want the folder, we don't want to break your Protobuf fix.
!cd custom_nodes && git clone https://github.com/Fannovel16/comfyui_controlnet_aux

print("✅ Files Downloaded! NOW you can restart.")

--2026-01-13 15:48:32--  https://huggingface.co/SG161222/Realistic_Vision_V5.1_noVAE/resolve/main/Realistic_Vision_V5.1_fp16-no-ema.safetensors
Resolving huggingface.co (huggingface.co)... 13.226.251.81, 13.226.251.112, 13.226.251.20, ...
Connecting to huggingface.co (huggingface.co)|13.226.251.81|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.gcp.cdn.hf.co/xet-bridge-us/64c744b3568845d756504c5f/d9e6165983f04ac09841060348d9c8b5d98552b2c337b18a3343ee917c4cfaa3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Realistic_Vision_V5.1_fp16-no-ema.safetensors%3B+filename%3D%22Realistic_Vision_V5.1_fp16-no-ema.safetensors%22%3B&Expires=1768322912&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY4MzIyOTEyfX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjRjNzQ0YjM1Njg4NDVkNzU2NTA0YzVmL2Q5ZTYxNjU5ODNmMDRhYzA5ODQxMDYwMzQ4ZDljOGI1ZDk4NTUyYjJjMzM3YjE4YTMzNDNlZTkxN2M0Y2ZhYTNcXD9yZXNwb2

In [5]:
# Download TAESD (Tiny VAE) - Corrected Links
!wget -c https://huggingface.co/madebyollin/taesd/resolve/main/taesd_decoder.safetensors -P ./models/vae/
!wget -c https://huggingface.co/madebyollin/taesd/resolve/main/taesd_encoder.safetensors -P ./models/vae/

--2026-01-13 13:36:28--  https://huggingface.co/madebyollin/taesd/resolve/main/taesd_decoder.safetensors
Resolving huggingface.co (huggingface.co)... 3.168.132.38, 3.168.132.40, 3.168.132.126, ...
Connecting to huggingface.co (huggingface.co)|3.168.132.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.gcp.cdn.hf.co/xet-bridge-us/64ba9fd90bd47e163a2664a2/4f0fbee2d1ef9357eaf4f10903745dabdd6b0f5e16924011c2789a4a16150e66?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27taesd_decoder.safetensors%3B+filename%3D%22taesd_decoder.safetensors%22%3B&Expires=1768314988&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY4MzE0OTg4fX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjRiYTlmZDkwYmQ0N2UxNjNhMjY2NGEyLzRmMGZiZWUyZDFlZjkzNTdlYWY0ZjEwOTAzNzQ1ZGFiZGQ2YjBmNWUxNjkyNDAxMWMyNzg5YTRhMTYxNTBlNjZcXD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=cCc-7E8WkJNgyeNWU8C-pMF8FppCmI5Gcg

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ComfyUI

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ComfyUI


### Run ComfyUI with cloudflared (Recommended Way)




In [3]:
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server --cpu

--2026-01-13 15:58:31--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb [following]
--2026-01-13 15:58:31--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/8a32f7c6-649c-4f0d-806d-e14c19d0786d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-01-13T16%3A37%3A51Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4d

### Run ComfyUI with localtunnel




In [ ]:
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

### Run ComfyUI with colab iframe (use only in case the previous way with localtunnel doesn't work)

You should see the ui appear in an iframe. If you get a 403 error, it's your firefox settings or an extension that's messing things up.

If you want to open it in another window use the link.

Note that some UI features like live image previews won't work because the colab iframe blocks websockets.

In [ ]:
import threading
import time
import socket
def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  from google.colab import output
  output.serve_kernel_port_as_iframe(port, height=1024)
  print("to open it in a window you can open this link here:")
  output.serve_kernel_port_as_window(port)

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server